Uncollateralized

Solution to Problem 1

In [1]:
import numpy as np
from math import *
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

In [2]:
# parameters for rates
a = 0.025   # t=0 forward interet rate curve f(0,t) = a + b*t
b = 0.002
sigma_r = 0.02 # short end volatility
c = 0.4 
kappa_2 = 0.05
rho_infinity = 0.4   #correlation between short end and long end
sigma_1 = sigma_r*c  # long end volatility 
v = sqrt(c**(-2)-1-2*(rho_infinity/c-1))
rho_x = (rho_infinity/c - 1)/v  #correlation between short and and a point between short and long end
sigma_2 = sigma_1*v

In [3]:
# parameters for swap
delta = 0.5     # exchange floating and fixed leg every half year
notional = 50000000
T = 10       # maturity of the swap
dt = 1/52     #d discretization
t_list = np.linspace(0,T,int(T/dt)+1)
h_list = [0.02,0.04,0.06,0.08]  # h values 

In [4]:
# other parameters
M = 100  # M paths
N = len(t_list) # N points
L = len(h_list) # L scenarios

In [5]:
# F function
def G(t,T):

    return np.array([[T - t], [1 / kappa_2 * (1-np.exp(-kappa_2*(T-t)))]])

def y_t(t):

    y_1 = sigma_1**2 * t + rho_x**2*sigma_1**2*sigma_2**2*(np.exp(2*kappa_2*t)-1)*1/(2*kappa_2)
    y_2 = sigma_1*sigma_2*rho_x*(sigma_2*1/(2*kappa_2)*(np.exp(kappa_2*t)-np.exp(-kappa_2*t))+ sigma_1/kappa_2*(1-np.exp(-kappa_2*t)))
    y_3 = (rho_x**2*sigma_1**2*sigma_2**2+sigma_2**2)*1/(2*kappa_2)*(1-np.exp(2*kappa_2*t))
    y_matrix = np.array([[y_1,y_2],[y_2,y_3]])
    
    return y_matrix

def F(t,T,xt):
    # F(t,T,xt) = P(t,T) given an xt dynamic
    def A(t,T):
        y_t_1 = y_t(t)
        a = y_t_1[0][0]
        b = y_t_1[0][1]
        c = y_t_1[1][0]
        d = y_t_1[1][1]
        
        return - ((T**2/2-t*T+t**2/2)*a - c/kappa_2**2*(1+np.exp(-kappa_2*(T-t))*(kappa_2*(t-T)-1)) + (T-t)*b/kappa_2
                  + b*(np.exp(-kappa_2*(T-t))-1)/kappa_2**2 + (d*np.exp(-2*kappa_2*(T-t))+d-2*d*np.exp(-kappa_2*(T-t)))/(2*kappa_2**2))
        
    def C(t,T):
        G_t_1 = G(t,T)
        return -G_t_1
    
    A1 = A(t,T)
    C1 = C(t,T)
    
    #f(0,T) = a + b*T
    first_term = exp(-a*T-b/2*T**2)/exp(-a*t-b/2*t**2)
    second_term = exp(A1+np.transpose(C1).dot(xt)[0])
    return first_term*second_term

In [6]:
# beta 
def beta(x1_lists,x2_lists):
    
    beta_lists = np.zeros((M,N))
    
    for j in range(M):
        r_list = np.zeros(N)
        r_list[0] = a + x1_lists[j][0] + x2_lists[j][0]
        r_integrand_list = np.zeros(N)
        beta_lists[j][0] = exp(r_integrand_list[0])
        for i in range(1,N):
            r_list[i] = a + b*i*dt + x1_lists[j][i] + x2_lists[j][i]
            r_integrand_list[i] = r_integrand_list[i-1] + (r_list[i] + r_list[i-1])*dt/2
            beta_lists[j][i] = exp(r_integrand_list[i])

    return beta_lists

In [7]:
# swap_pv
def swap_pv_t(swap_type, h, delta, notional,t, T, xt, beta_list, dt):
    
    if t == T:
        return 0
    
    else:
        Ti_list = np.linspace(0,T,int(T//delta)+1)
        if swap_type == 'payer':
            sign = -1
        else:
            sign = 1

        for i in range(len(Ti_list)):
            
            if Ti_list[i] == t:
                T0 = Ti_list[i]
                Ti_new_list = Ti_list[i+1:]
                break
                
            elif Ti_list[i] > t:
                T0 = Ti_list[i-1]
                Ti_new_list = Ti_list[i:]
                break

        pv = F(t,Ti_new_list[-1],xt) - beta_list[int(t//dt)]/beta_list[int(T0//dt)]
        
        for i in range(len(Ti_new_list)):
            Ti = Ti_new_list[i]
            p_t_Ti = F(t,Ti, xt)
            pv += (h*delta)* p_t_Ti

        return pv*sign*notional

In [8]:
# MC_path function to generate one x1 path and one x2 path
def MC_x(T,dt,M,N,kappa_2,sigma_1,sigma_2,rho_x):
    
    x1_lists = np.zeros((M, N))
    x2_lists = np.zeros((M, N))
    
    kappa = np.array([[0,0],[0,kappa_2]])
    sigma_x = np.array([[sigma_1,0],[0,sigma_2]])
    ones = np.array([[1],[1]])
    
    for j in tqdm(range(M)):
        dW1 = np.random.normal(0,sqrt(dt),N-1)
        dW0 = np.random.normal(0,sqrt(dt),N-1)
        dW2 = rho_x*dW1 + sqrt(1-rho_x**2)*dW0

        for i in range(N-1):
            t = t_list[i]
            x_1 = x1_lists[j][i]
            x_2 = x2_lists[j][i]
            x_t = np.array([[x_1],[x_2]])
            dW = np.array([[dW1[i]],[dW2[i]]])
            y_t1 = y_t(t)
            dx = (y_t1.dot(ones) - kappa.dot(x_t))*dt + sigma_x.dot(dW)
            x1_lists[j][i+1] = x_1+dx[0][0]
            x2_lists[j][i+1] = x_2+dx[1][0]
        
    return x1_lists,x2_lists

In [9]:
# generate paths for x1 and x2
x1_lists, x2_lists = MC_x(T,dt,M,N,kappa_2,sigma_1,sigma_2,rho_x)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 91.51it/s]


In [10]:
# generate pv given h
def pv_generation(x1_lists, x2_lists, delta, notional, t_list, T, dt, h):
    
    m = len(x1_lists)
    n = len(t_list)
    
    beta_lists = beta(x1_lists,x2_lists)
    rec_swap_pv = np.zeros((m,n))
    pay_swap_pv = np.zeros((m,n))

    for j in tqdm(range(m)):
        
        beta_list = beta_lists[j]

        for i in range(n):

            t = t_list[i]
            x_1 = x1_lists[j][i]
            x_2 = x2_lists[j][i]
            xt = np.array([[x_1],[x_2]])

            rec_swap_pv_t = swap_pv_t('receiver', h, delta, notional, t, T, xt, beta_list, dt)
            rec_swap_pv[j][i] = rec_swap_pv_t
            pay_swap_pv[j][i] = -rec_swap_pv_t
                
    return rec_swap_pv, pay_swap_pv

In [11]:
# PV genearation
rec_swap_pvs = []  # store the pv_lists for different h
pay_swap_pvs = []
for h in h_list:
    rec_swap_pv, pay_swap_pv = pv_generation(x1_lists, x2_lists, delta, notional, t_list, T, dt, h)
    rec_swap_pvs.append(rec_swap_pv)
    pay_swap_pvs.append(pay_swap_pv)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:21<00:00,  4.70it/s]


In [12]:
# generate exposure given pv and method
def exp_generation(pv_lists, method):
    
    m = len(pv_lists)
    n = len(pv_lists[0])
    
    if method == 'uncollateralized':
        exp_lists = np.where(pv_lists>0, pv_lists, 0)

    elif method == 'variation margin': 
        #full variation margin (MPoR is 2 weeks)
        exp_lists = np.zeros((m, n))
        
        for j in range(m):
            for i in range(2,n):
                exp_lists[j][i] = np.maximum(pv_lists[j][i] - pv_lists[j][i-2], 0)
            
            for i in range(2):
                exp_lists[j][i] = exp_lists[j][2]  
                
    else:
        assert False, "This method are not defined!"
        
    return exp_lists

In [13]:
# EXP generation
rec_swap_exps = []
pay_swap_exps = []

for rec_swap_pv in rec_swap_pvs:
    rec_swap_exps.append(exp_generation(rec_swap_pv, 'uncollateralized'))

for pay_swap_pv in pay_swap_pvs:
    pay_swap_exps.append(exp_generation(pay_swap_pv, 'uncollateralized'))

In [14]:
# generate expected exposure given exposure
def ee_generation(exp_lists):
    ee_list = np.mean(exp_lists, axis = 0)
    return ee_list

In [15]:
# EE generation
rec_swap_ees = []
pay_swap_ees = []

for rec_swap_exp in rec_swap_exps:
    rec_swap_ees.append(ee_generation(rec_swap_exp))

for pay_swap_exp in pay_swap_exps:
    pay_swap_ees.append(ee_generation(pay_swap_exp))

In [16]:
rec_swap_ees

[array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.10045169e+02, 0.00000000e+00, 4.01332239e+04,
        5.10137833e+04, 8.28372444e+04, 6.27341862e+04, 6.26664838e+04,
        7.57060077e+04, 8.77678182e+04, 1.12950609e+05, 1.50542547e+05,
        1.56037361e+05, 1.55373371e+05, 1.80383839e+05, 1.82126393e+05,
        2.28012980e+05, 2.51374804e+05, 2.21703228e+05, 2.45681455e+05,
        2.35347556e+05, 2.14785674e+05, 3.02641121e+05, 3.62090928e+05,
        3.53305753e+05, 3.62872022e+05, 3.49829771e+05, 3.29659571e+05,
        3.45468875e+05, 3.05118618e+05, 3.30706201e+05, 3.64817384e+05,
        3.70896155e+05, 3.72284901e+05, 3.09000308e+05, 3.73052035e+05,
        3.44404037e+05, 4.15254032e+05, 4.32023550e+05, 4.85512698e+05,
        5.31378008e+05, 5.54980694e+05, 5.75312598e+05, 5.93053901e+05,
        5.29191650e+05, 5.16732554e+05, 5.93436404e+05, 6.621648

In [17]:
# generate EPE 
def epe_calculation(ee_list, dt):
    
    n = len(ee_list)
    epe_list = np.zeros(n)
    epe_list[0] = ee_list[0] # epe starts from EE[0]
    integrand = 0      
    
    for i in range(n-1):
        integrand += dt/2*(ee_list[i] + ee_list[i+1])             
        epe_list[i+1] = integrand/(dt*(i+1))
        
    return epe_list

In [18]:
# EPE generation
rec_swap_epes = []
pay_swap_epes = []

for rec_swap_ee in rec_swap_ees:
    rec_swap_epes.append(epe_calculation(rec_swap_ee, dt))

for pay_swap_ee in pay_swap_ees:
    pay_swap_epes.append(epe_calculation(pay_swap_ee, dt))

In [19]:
# EPE(1)
epe_1s = np.zeros((2, L))

for k in range(L):
    epe_1s[0][k] = rec_swap_epes[k][52]    # 0 represents rec  when T = 1, it is the 52nd week
    epe_1s[1][k] = pay_swap_epes[k][52] 

In [20]:
# generate EE* given EE
def ee_star_generation(ee_list):
    
    n = len(ee_list)
    ee_star_list = np.zeros(n)
    
    for i in range(1, n):
        ee_star_list[i] = max(ee_list[i], ee_star_list[i-1])
        
    return ee_star_list

In [21]:
# EE* generation and EEPE generation
rec_swap_ee_stars = []
pay_swap_ee_stars = []

for rec_swap_ee in rec_swap_ees:
    rec_swap_ee_stars.append(ee_star_generation(rec_swap_ee))
    
for pay_swap_ee in pay_swap_ees:
    pay_swap_ee_stars.append(ee_star_generation(pay_swap_ee))
    
rec_swap_eepes = []
pay_swap_eepes = []

for rec_swap_ee_star in rec_swap_ee_stars:
    rec_swap_eepes.append(epe_calculation(rec_swap_ee_star, dt))

for pay_swap_ee_star in pay_swap_ee_stars:
    pay_swap_eepes.append(epe_calculation(pay_swap_ee_star, dt))

In [22]:
# EEPE(1)
eepe_1s = np.zeros((2, L))
                  
for k in range(L):
    eepe_1s[0][k] = rec_swap_eepes[k][52]   
    eepe_1s[1][k] = pay_swap_eepes[k][52] 

In [23]:
# generate EAD given PV 
# EAD under Basel 3 considering CVA
# EAD = 1.06 * max(alpha*EEPE(1)-CVAu, 0)
def cva_calculation(exp_lists, x1_lists, x2_lists, t_list, lambda_a, r_a):
    
    beta_lists = beta(x1_lists,x2_lists)
    n = len(t_list)    
    pve_lists = exp_lists/beta_lists
    pvee_list = ee_generation(pve_lists)
    
    cva = 0
    
    for i in range(1,n):
        cva += -(1-r_a)*(exp(-lambda_a*t_list[i])-exp(-lambda_a*t_list[i-1]))*pvee_list[i]
    
    return cva

def ead_calculation(cva, alpha_basel, eepe_1):
      
    ead = 1.06* max(alpha_basel * eepe_1 - cva, 0)
    
    return ead

In [24]:
# EAD
lambda_a = 0.045
r_a = 0.4    # Let's assume the recovery rate for A is still 40% as before
alpha_basel = 1.4

cvas = np.zeros((2,L))

for k in range(L):
    cvas[0][k] = cva_calculation(rec_swap_exps[k], x1_lists, x2_lists, t_list, lambda_a, r_a)
    cvas[1][k] = cva_calculation(pay_swap_exps[k], x1_lists, x2_lists, t_list, lambda_a, r_a)

eads = np.zeros((2, L))
                  
for k in range(L):
    eads[0][k] = ead_calculation(cvas[0][k], alpha_basel, eepe_1s[0][k])
    eads[1][k] = ead_calculation(cvas[1][k], alpha_basel, eepe_1s[1][k])

In [25]:
# generate M given EE*, P_list, EE

def m_calculation(ee_list, ee_star_list, P_list, t_list):
    
    t1_list = t_list[52:]
    ee_discounted_list = np.zeros(len(t1_list))
    
    for i in range(len(t1_list)):
        t1 = t1_list[i]
        ee_discounted_list[i] = ee_list[52+i]*P_list[52+i]
    
    t2_list = t_list[:53]
    ee_star_discounted_list = np.zeros(len(t2_list))
    
    for i in range(len(t2_list)):
        t2 = t2_list[i]
        ee_star_discounted_list[i] = ee_star_list[i]*P_list[i]
   
    numerator_integrand = 0
    for i in range(len(ee_discounted_list)-1):
        numerator_integrand += dt/2*(ee_discounted_list[i] + ee_discounted_list[i+1])
    
    denominator_integrand = 0
    for i in range(len(ee_star_discounted_list)-1):
        denominator_integrand += dt/2*(ee_star_discounted_list[i] + ee_star_discounted_list[i+1])
    
    term = 1 + numerator_integrand/denominator_integrand
    
    return max(1,min(5,term))

In [26]:
# generate P_list.  P(t) is E(1/beta(t))
beta_lists = beta(x1_lists, x2_lists)
P_list = np.mean(1/beta_lists, axis = 0)

Ms = np.zeros((2, L))

for k in range(L):
    
    Ms[0][k] = m_calculation(rec_swap_ees[k], rec_swap_ee_stars[k], P_list, t_list)
    Ms[1][k] = m_calculation(pay_swap_ees[k], pay_swap_ee_stars[k], P_list, t_list)

<ipython-input-25-cd4159068b85>:27: RuntimeWarning: divide by zero encountered in double_scalars
  term = 1 + numerator_integrand/denominator_integrand


In [27]:
p1_dict = {'EPE(1)_rec':epe_1s[0],
           'EEPE(1)_rec':eepe_1s[0],
           'EAD_rec':eads[0],
           'M_rec':Ms[0],
           'EPE(1)_pay':epe_1s[1],
           'EEPE(1)_pay':eepe_1s[1],
           'EAD_pay':eads[1],
           'M_pay':Ms[1]}
p1_df = pd.DataFrame(p1_dict,index = h_list)
print('Uncollaterized EPE(1),EEPE(1),EAD, M')
p1_df

Uncollaterized EPE(1),EEPE(1),EAD, M


,EPE(1)_rec,EEPE(1)_rec,EAD_rec,M_rec,EPE(1)_pay,EEPE(1)_pay,EAD_pay,M_pay
0.02,2.274144e+05,2.355372e+05,5.295327e+04,5.0,6.449669e+06,6.532072e+06,8.755533e+06,5.0
0.04,3.031634e+06,3.066464e+06,3.768600e+06,5.0,9.021175e+05,9.146335e+05,9.919398e+05,5.0
0.06,1.052449e+07,1.082893e+07,1.446529e+07,5.0,4.319894e+04,5.248737e+04,0.000000e+00,5.0
0.08,1.883306e+07,1.930977e+07,2.608102e+07,5.0,0.000000e+00,0.000000e+00,0.000000e+00,5.0


Solution to Problem 2

In [28]:
PD =  0.005
LGD = 0.6

In [29]:
# k
def k_calculation(M, PD):
    return (1+(M - 2.5)*(0.11852 - 0.05478*log(PD))**2)/(1 - 1.5*(0.11852 - 0.05478*log(PD))**2)

Ks = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        Ks[i][k] = k_calculation(Ms[i][k], PD)

In [30]:
# RC
rc_calculation = lambda EAD,RW: EAD*RW

def rw_calculation(PD, K, LGD):
    
    q = 0.001
    
    rho_p = lambda p: 0.24 - 0.12*(1-exp(-50*p))
    
    term = norm.cdf((norm.ppf(PD) - sqrt(rho_p(PD))*norm.ppf(q))/sqrt(1-rho_p(PD))) - PD
    
    return LGD*term*K

RCs = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        RCs[i][k] = rc_calculation(eads[i][k], rw_calculation(PD, Ks[i][k], LGD))

In [31]:
p2_dict = {'k_rec': Ks[0],
           'RC_rec': RCs[0],
           'k_pay': Ks[1],
           'RC_pay': RCs[1]}
p2_df = pd.DataFrame(p2_dict,index = h_list)
print('Uncollaterized k, RC')
p2_df

Uncollaterized k, RC


,k_rec,RC_rec,k_pay,RC_pay
0.02,1.891875,5.574335e+03,1.891875,921685.794435
0.04,1.891875,3.967166e+05,1.891875,104420.468507
0.06,1.891875,1.522746e+06,1.891875,0.000000
0.08,1.891875,2.745521e+06,1.891875,0.000000


Full Variation Margin

Solution to Problem 3

In [32]:
#EPE(1)
rec_swap_exps_fvm = []
pay_swap_exps_fvm = []

for rec_swap_pv in rec_swap_pvs:
    rec_swap_exps_fvm.append(exp_generation(rec_swap_pv, 'variation margin'))

for pay_swap_pv in pay_swap_pvs:
    pay_swap_exps_fvm.append(exp_generation(pay_swap_pv, 'variation margin'))
    
rec_swap_ees_fvm = []
pay_swap_ees_fvm = []

for rec_swap_exp_fvm in rec_swap_exps_fvm:
    rec_swap_ees_fvm.append(ee_generation(rec_swap_exp_fvm))

for pay_swap_exp_fvm in pay_swap_exps_fvm:
    pay_swap_ees_fvm.append(ee_generation(pay_swap_exp_fvm))
    
rec_swap_epes_fvm = []
pay_swap_epes_fvm = []

for rec_swap_ee_fvm in rec_swap_ees_fvm:
    rec_swap_epes_fvm.append(epe_calculation(rec_swap_ee_fvm, dt))

for pay_swap_ee_fvm in pay_swap_ees_fvm:
    pay_swap_epes_fvm.append(epe_calculation(pay_swap_ee_fvm, dt))

epe_1s_fvm = np.zeros((2, L))

for k in range(L):
    epe_1s_fvm[0][k] = rec_swap_epes_fvm[k][52]
    epe_1s_fvm[1][k] = pay_swap_epes_fvm[k][52] 

In [33]:
# EEPE(1)
rec_swap_ee_stars_fvm = []
pay_swap_ee_stars_fvm = []

for rec_swap_ee_fvm in rec_swap_ees_fvm:
    rec_swap_ee_stars_fvm.append(ee_star_generation(rec_swap_ee_fvm))
    
for pay_swap_ee_fvm in pay_swap_ees_fvm:
    pay_swap_ee_stars_fvm.append(ee_star_generation(pay_swap_ee_fvm))
    
rec_swap_eepes_fvm = []
pay_swap_eepes_fvm = []

for rec_swap_ee_star_fvm in rec_swap_ee_stars_fvm:
    rec_swap_eepes_fvm.append(epe_calculation(rec_swap_ee_star_fvm, dt))

for pay_swap_ee_star_fvm in pay_swap_ee_stars_fvm:
    pay_swap_eepes_fvm.append(epe_calculation(pay_swap_ee_star_fvm, dt))

eepe_1s_fvm = np.zeros((2, L))

for k in range(L):
    eepe_1s_fvm[0][k] = rec_swap_eepes_fvm[k][52]   
    eepe_1s_fvm[1][k] = pay_swap_eepes_fvm[k][52] 

In [34]:
# EAD
cvas_fvm = np.zeros((2,L))

for k in range(L):
    cvas_fvm[0][k] = cva_calculation(rec_swap_exps_fvm[k], x1_lists, x2_lists, t_list, lambda_a, r_a)
    cvas_fvm[1][k] = cva_calculation(pay_swap_exps_fvm[k], x1_lists, x2_lists, t_list, lambda_a, r_a)

eads_fvm = np.zeros((2, L))
                  
for k in range(L):
    eads_fvm[0][k] = ead_calculation(cvas_fvm[0][k], alpha_basel, eepe_1s_fvm[0][k])
    eads_fvm[1][k] = ead_calculation(cvas_fvm[1][k], alpha_basel, eepe_1s_fvm[1][k])

In [35]:
# M
Ms_fvm = np.zeros((2, L))

for k in range(L):
    
    Ms_fvm[0][k] = m_calculation(rec_swap_ees_fvm[k], rec_swap_ee_stars_fvm[k], P_list, t_list)
    Ms_fvm[1][k] = m_calculation(pay_swap_ees_fvm[k], pay_swap_ee_stars_fvm[k], P_list, t_list)

In [36]:
# k
Ks_fvm = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        Ks_fvm[i][k] = k_calculation(Ms_fvm[i][k], PD)

In [37]:
# RC
RCs_fvm = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        RCs_fvm[i][k] = rc_calculation(eads_fvm[i][k], rw_calculation(PD, Ks_fvm[i][k], LGD))

In [38]:
p3_dict = {'EPE(1)_rec':epe_1s_fvm[0],
           'EEPE(1)_rec':eepe_1s_fvm[0],
           'EAD_rec':eads_fvm[0],
           'M_rec':Ms_fvm[0],
           'k_rec':Ks_fvm[0],
           'RC_rec':RCs_fvm[0],
           'EPE(1)_pay':epe_1s_fvm[1],
           'EEPE(1)_pay':eepe_1s_fvm[1],
           'EAD_pay':eads_fvm[1],
           'M_pay':Ms_fvm[1],
           'k_pay':Ks_fvm[1],
           'RC_pay':RCs_fvm[1]}
print('Full Variation Margin EPE(1), EEPE(1), EAD, M, k, RC')
p3_df = pd.DataFrame(p3_dict,index = h_list)
p3_df

Full Variation Margin EPE(1), EEPE(1), EAD, M, k, RC


,EPE(1)_rec,EEPE(1)_rec,EAD_rec,M_rec,k_rec,RC_rec,EPE(1)_pay,EEPE(1)_pay,EAD_pay,M_pay,k_pay,RC_pay
0.02,497540.145721,620195.933864,8.485823e+05,5.000000,1.891875,89329.370979,480881.138980,5.961131e+05,8.162480e+05,4.869560,1.862791,84604.630399
0.04,543912.696625,691577.439484,9.524104e+05,4.625627,1.808401,95835.594127,541885.342159,7.915003e+05,1.097793e+06,4.263151,1.727581,105527.719014
0.06,593436.576857,762958.945103,1.055210e+06,4.362012,1.749624,102728.644435,606040.874666,1.038512e+06,1.454920e+06,3.793687,1.622905,131383.253727
0.08,645000.925432,834341.837332,1.157205e+06,4.185172,1.710194,110119.271968,672236.875516,1.306505e+06,1.842376e+06,3.490832,1.555378,159449.083154


Solution to Problem 4

In [39]:
# To remove the spikes at payment date, we suggest that the EE(t) should be equal to (EE(t-1) + EE(t+1))/2 
# when t is the payment date
def remove_spike(ee_list):
    
    n = len(ee_list)
    
    for i in range(1,n-1):
        if i%26 == 0:
            ee_list[i] = (ee_list[i-1] + ee_list[i+1])/2
    
    return ee_list

In [40]:
# EE
rec_swap_ees_fvm_w = []
pay_swap_ees_fvm_w = []

for rec_swap_ee_fvm in rec_swap_ees_fvm:
    rec_swap_ees_fvm_w.append(remove_spike(rec_swap_ee_fvm))
    
for pay_swap_ee_fvm in pay_swap_ees_fvm:
    pay_swap_ees_fvm_w.append(remove_spike(pay_swap_ee_fvm))

In [41]:
# EPE(1)
rec_swap_epes_fvm_w = []
pay_swap_epes_fvm_w = []

for rec_swap_ee_fvm_w in rec_swap_ees_fvm_w:
    rec_swap_epes_fvm_w.append(epe_calculation(rec_swap_ee_fvm_w, dt))

for pay_swap_ee_fvm_w in pay_swap_ees_fvm_w:
    pay_swap_epes_fvm_w.append(epe_calculation(pay_swap_ee_fvm_w, dt))

epe_1s_fvm_w = np.zeros((2, L))

for k in range(L):
    epe_1s_fvm_w[0][k] = rec_swap_epes_fvm_w[k][52]
    epe_1s_fvm_w[1][k] = pay_swap_epes_fvm_w[k][52] 

In [42]:
# EEPE(1)
rec_swap_ee_stars_fvm_w = []
pay_swap_ee_stars_fvm_w = []

for rec_swap_ee_fvm_w in rec_swap_ees_fvm_w:
    rec_swap_ee_stars_fvm_w.append(ee_star_generation(rec_swap_ee_fvm_w))
    
for pay_swap_ee_fvm_w in pay_swap_ees_fvm_w:
    pay_swap_ee_stars_fvm_w.append(ee_star_generation(pay_swap_ee_fvm_w))
    
rec_swap_eepes_fvm_w = []
pay_swap_eepes_fvm_w = []

for rec_swap_ee_star_fvm_w in rec_swap_ee_stars_fvm_w:
    rec_swap_eepes_fvm_w.append(epe_calculation(rec_swap_ee_star_fvm_w, dt))

for pay_swap_ee_star_fvm_w in pay_swap_ee_stars_fvm_w:
    pay_swap_eepes_fvm_w.append(epe_calculation(pay_swap_ee_star_fvm_w, dt))

eepe_1s_fvm_w = np.zeros((2, L))

for k in range(L):
    eepe_1s_fvm_w[0][k] = rec_swap_eepes_fvm_w[k][52]   
    eepe_1s_fvm_w[1][k] = pay_swap_eepes_fvm_w[k][52] 

In [43]:
# EAD
eads_fvm_w = np.zeros((2, L))
                  
for k in range(L):
    eads_fvm_w[0][k] = ead_calculation(cvas_fvm[0][k], alpha_basel, eepe_1s_fvm_w[0][k])
    eads_fvm_w[1][k] = ead_calculation(cvas_fvm[1][k], alpha_basel, eepe_1s_fvm_w[1][k])

In [44]:
# M
Ms_fvm_w = np.zeros((2, L))

for k in range(L):
    Ms_fvm_w[0][k] = m_calculation(rec_swap_ees_fvm_w[k], rec_swap_ee_stars_fvm_w[k], P_list, t_list)
    Ms_fvm_w[1][k] = m_calculation(pay_swap_ees_fvm_w[k], pay_swap_ee_stars_fvm_w[k], P_list, t_list)

In [45]:
# k
Ks_fvm_w = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        Ks_fvm_w[i][k] = k_calculation(Ms_fvm_w[i][k], PD)

In [46]:
# RC
RCs_fvm_w = np.zeros((2, L))

for i in range(2):
    for k in range(L):
        RCs_fvm_w[i][k] = rc_calculation(eads_fvm_w[i][k], rw_calculation(PD, Ks_fvm_w[i][k], LGD))

In [47]:
p4_dict = {'EPE(1)_rec':epe_1s_fvm_w[0],
           'EEPE(1)_rec':eepe_1s_fvm_w[0],
           'EAD_rec':eads_fvm_w[0],
           'M_rec':Ms_fvm_w[0],
           'k_rec':Ks_fvm_w[0],
           'RC_rec':RCs_fvm_w[0],
           'EPE(1)_pay':epe_1s_fvm_w[1],
           'EEPE(1)_pay':eepe_1s_fvm_w[1],
           'EAD_pay':eads_fvm_w[1],
           'M_pay':Ms_fvm_w[1],
           'k_pay':Ks_fvm_w[1],
           'RC_pay':RCs_fvm_w[1]}
print('Spike Removed Full Variation Margin EPE(1),EEPE(1),EAD, M, k, RC')
p4_df = pd.DataFrame(p4_dict,index = h_list)
p4_df

Spike Removed Full Variation Margin EPE(1),EEPE(1),EAD, M, k, RC


,EPE(1)_rec,EEPE(1)_rec,EAD_rec,M_rec,k_rec,RC_rec,EPE(1)_pay,EEPE(1)_pay,EAD_pay,M_pay,k_pay,RC_pay
0.02,495308.994911,620195.933864,8.485823e+05,4.924033,1.874937,88529.592611,482164.930498,5.961131e+05,8.162480e+05,4.855143,1.859576,84458.628084
0.04,545803.740702,691577.439484,9.524104e+05,4.598669,1.802391,95517.051298,540164.064295,7.899989e+05,1.095565e+06,4.216116,1.717093,104674.226396
0.06,598090.047023,762958.945103,1.055210e+06,4.376334,1.752817,102916.149575,599954.758622,1.034271e+06,1.448627e+06,3.722218,1.606969,129530.475876
0.08,651741.761243,834341.837332,1.157205e+06,4.222134,1.718435,110649.936402,661110.860848,1.299067e+06,1.831338e+06,3.397949,1.534668,156383.454212


Solution to Problem 5

In [48]:
h_par = 0.0347
rho_l = 0 
rho_s = 0
kappa_lambda = 0.04
sigma_lambda = 0.01
a_lambda = 0.045

In [49]:
def MC_lambda(a, kappa, sigma, M, N, dt):
    lamb = np.zeros((M, N))
    
    for j in range(M):
        dW = np.random.normal(0,sqrt(dt),N-1)
        x = 0 
        lamb[j][0] = a + x
        
        for i in range(N-1):
            t = dt*i
            y = sigma**2*(1-e**(-2*kappa*t))/2*kappa
            dx = (y - kappa*x)*dt + sigma*dW[i]
            x += dx
            lamb[j][i+1] = a + x
            
    return lamb

In [50]:
# generate lambda_a MC path
lambda_a_lists = MC_lambda(a_lambda, kappa_lambda, sigma_lambda, M, N, dt)

In [51]:
def r_calculation(x1_lists,x2_lists, a, b, M, N):
    
    r_lists = np.zeros((M,N))
    
    for j in range(M):
        for i in range(N):
            r_lists[j][i] = a+ b*i*dt + x1_lists[j][i] + x2_lists[j][i]
            
    return r_lists

In [52]:
# generate r_lists
r_lists = r_calculation(x1_lists, x2_lists, a, b, M, N)

In [53]:
def alpha(r_lists, lambda_lists):
    
    alpha_lists = np.zeros((M, N))
    integrand_lists = np.zeros((M, N))
    
    for j in range(M):
        alpha_lists[j][0] = exp(-integrand_lists[j][0])
        
        for i in range(1, N):
            integrand_lists[j][i] = integrand_lists[j][i-1] + (r_lists[j][i] + lambda_lists[j][i]+ r_lists[j][i-1] + lambda_lists[j][i-1])*dt/2
            alpha_lists[j][i] = exp(-integrand_lists[j][i])
            
    return alpha_lists

In [54]:
# generate alpha_lists
alpha_lists = alpha(r_lists, lambda_a_lists)

In [55]:
# generate swap_exp when h is h_par
rec_swap_pv_hpar, pay_swap_pv_hpar = pv_generation(x1_lists, x2_lists, delta, notional, t_list, T, dt, h_par)
rec_swap_exp_hpar = exp_generation(rec_swap_pv_hpar, 'uncollateralized')
pay_swap_exp_hpar = exp_generation(pay_swap_pv_hpar, 'uncollateralized')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.81it/s]


In [56]:
def cva_revised(t_list, exp_lists, alpha_lists, lambda_lists, r_a):
    
    total = exp_lists*alpha_lists*lambda_lists
    total_expectation = np.mean(total, axis = 0)
    
    cva = 0
    for i in range(1,N):
        cva += (1-r_a)*dt*(total_expectation[i-1]+total_expectation[i])/2
    
    return cva

In [57]:
cva_revised_receiver_p5 = cva_revised(t_list, rec_swap_exp_hpar, alpha_lists, lambda_a_lists, r_a)
cva_revised_payer_p5 = cva_revised(t_list, pay_swap_exp_hpar, alpha_lists, lambda_a_lists, r_a)

In [58]:
print('For receiver, the CVA is', cva_revised_receiver_p5)
print('For payer, the CVA is', cva_revised_payer_p5)

For receiver, the CVA is 612025.5533717498
For payer, the CVA is 432604.9834189023


Solution to Problem 6

In [59]:
rho_l = 0.2 
rho_s = 0.4

In [60]:
rho_z_1 = rho_l
rho_z_2 = rho_s*sqrt(v**2 + 1 + 2*rho_x*v) - rho_z_1*v

In [61]:
def MC(a_lambda, kappa_lambda, sigma_lambda, M, N, dt, T, kappa_2, sigma_1, sigma_2, rho_x):
    
    x1_lists = np.zeros((M, N))
    x2_lists = np.zeros((M, N))
    lamb_lists = np.zeros((M, N))
    
    kappa = np.array([[0,0],[0,kappa_2]])
    sigma_x = np.array([[sigma_1,0],[0,sigma_2]])
    ones = np.array([[1],[1]])
    
    for j in tqdm(range(M)):
        dZ = np.random.normal(0,sqrt(dt),N-1)   # for lambda
        dZ1 = np.random.normal(0,sqrt(dt),N-1)
        dZ2 = np.random.normal(0,sqrt(dt),N-1)
        
        dW1 = rho_z_1*dZ + sqrt(1-rho_z_1**2)*dZ1  # for x1
        dW2 = rho_z_2*dZ + (rho_x - rho_z_1*rho_z_2)/sqrt(1-rho_z_1**2)*dZ1 + sqrt(1 - rho_z_2**2 - ((rho_x - rho_z_1*rho_z_2)/sqrt(1-rho_z_1**2))**2)*dZ2      # for x2
                                                                                   
        x = 0 
        lamb_lists[j][0] = a_lambda + x
        
        for i in range(N-1):
            t = dt*i
            y = sigma_lambda**2*(1-e**(-2*kappa_lambda*t))/2*kappa_lambda
            dx = (y - kappa_lambda*x)*dt + sigma_lambda*dZ[i]
            x += dx
            lamb_lists[j][i+1] = a_lambda + x
            
            x_1 = x1_lists[j][i]
            x_2 = x2_lists[j][i]
            x_t = np.array([[x_1],[x_2]])
            dW = np.array([[dW1[i]],[dW2[i]]])
            y_t1 = y_t(t)
            dx = (y_t1.dot(ones) - kappa.dot(x_t))*dt + sigma_x.dot(dW)
            x1_lists[j][i+1] = x_1+dx[0][0]
            x2_lists[j][i+1] = x_2+dx[1][0]
        
    return x1_lists, x2_lists, lamb_lists

In [62]:
x1_lists_new, x2_lists_new, lamb_lists_new = MC(a_lambda, kappa_lambda, sigma_lambda, M, N, dt, T, kappa_2, sigma_1, sigma_2, rho_x)
r_lists_new = r_calculation(x1_lists_new, x2_lists_new, a, b, M, N)
alpha_lists_new = alpha(r_lists_new, lamb_lists_new)
rec_swap_pv_hpar_new, pay_swap_pv_hpar_new = pv_generation(x1_lists_new, x2_lists_new, delta, notional, t_list, T, dt, h_par)
rec_swap_exp_hpar_new = exp_generation(rec_swap_pv_hpar_new, 'uncollateralized')
pay_swap_exp_hpar_new = exp_generation(pay_swap_pv_hpar_new, 'uncollateralized')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.77it/s]


In [63]:
cva_revised_receiver_p6 = cva_revised(t_list, rec_swap_exp_hpar_new, alpha_lists_new, lamb_lists_new, r_a)
cva_revised_payer_p6 = cva_revised(t_list, pay_swap_exp_hpar_new, alpha_lists_new, lamb_lists_new, r_a)

In [64]:
print('For receiver, the CVA is', cva_revised_receiver_p6)
print('For payer, the CVA is', cva_revised_payer_p6)

For receiver, the CVA is 509707.27802968177
For payer, the CVA is 533390.1507489436


In [65]:
# For bi-laterla, it's a zero-sum game. So one party's reduction in CVA must be another party's game. 
# This is why we can see one value increase whereas the other one decrease. 
# For here, the payer receives the floating legs. With wrong-way risk, the exposure for payer to the receiver increases,
# due to the correlation between interest rates and exposure. So we can see an increase in payer's CVA, which at the 
# same time, is reciever's CVA reduction.